# Laboratorio 6 Grupal "Aprendizaje por Refuerzo".
#### Nombres:
####          - Piza Nava Vladimir.
####          - Aramayo Valdez Joaquin.
####          - Viza Hoyos Maria Belen.
####          - Mendoza Ovando Carlos Saul.
####          - Solorzano Diego.
#### Link GitHub: https://github.com/Joaco15045F/InteligenciaArtificial/blob/main/Laboratorio6%20grupal%20AxR/Lab6_BipedalWalker.ipynb
#### Link info BipedalWalker: https://www.gymlibrary.dev/environments/box2d/bipedal_walker/

##### En este proyecto se desarrollará un agente capaz de aprender a caminar en un entorno de Bipedal Walker utilizando el algoritmo de Q-Learning. A través de la interacción con el entorno, el agente irá ajustando su comportamiento mediante la actualización de su tabla Q, que representa las recompensas esperadas por cada acción en cada estado. El objetivo es entrenar al agente para que logre caminar de manera eficiente, maximizando su recompensa acumulada mediante una política de explotación, es decir, eligiendo las mejores acciones basadas en lo aprendido.


### Importación de librerías
#### Se importan las librerías necesarias para trabajar con operaciones numéricas (NumPy), entornos de aprendizaje por refuerzo (gymnasium), visualización de gráficos (matplotlib.pyplot) y diccionarios con valores predeterminados (defaultdict).



In [ ]:
import numpy as np # type: ignore
import gymnasium as gym # type: ignore
import matplotlib.pyplot as plt # type: ignore
from collections import defaultdict

### Inicialización del entorno
#### Se crea e inicializa el entorno de simulación "BipedalWalker-v3" utilizando la librería gymnasium, que simula un robot bípedo en un escenario de caminata.

In [2]:
# Inicializar el entorno
env = gym.make("BipedalWalker-v3")

### Parámetros de Q-learning
#### Se configuran los parámetros principales del algoritmo de Q-learning: alpha (tasa de aprendizaje) que ajusta la rapidez con la que se aprende de nuevas experiencias, gamma (factor de descuento) que controla la importancia de las recompensas futuras, epsilon (tasa de exploración) que determina la probabilidad de explorar acciones nuevas, y n_bins (número de divisiones) para discretizar las observaciones del entorno.

In [ ]:
# Parámetros de Q-learning
# Tasa de aprendizaje
alpha = 0.1       # cerca de 0 el agente aprende poco y cerca de 1 el agente aprende mucho
# Factor de descuento
gamma = 0.99      # cerca de 0 el agente considera solo recompensas inmediatas y cerca de 1 considera recompensas a largo plazo
# Tasa de exploración 
epsilon = 0.1     # cerca de 0 el agente prefiere explotar y cerca de 1 el agente prefiere explorar
# Número de episodios
n_bins = 5        # Número de divisiones para discretización en cada dimensión

### Discretización del espacio de observación
#### La función discretize_obs convierte las observaciones continuas del entorno en un formato discreto. Para ello, crea intervalos (bins) en cada dimensión del espacio de observación utilizando n_bins divisiones. Luego, asigna a cada observación el índice correspondiente dentro de su intervalo mediante np.digitize, facilitando la representación de las observaciones de manera discreta que puede ser utilizada por el algoritmo de Q-learning.

In [ ]:
# cambia de la observaciones de continuo a discreto
def discretize_obs(obs):
    bins = [np.linspace(-5, 5, n_bins) for _ in range(len(obs))] # Discretización uniforme
    discretized = tuple(np.digitize(o, bins[i]) for i, o in enumerate(obs)) # Discretizar cada dimensión
    return discretized # Estado discretizado

### Creación de conjunto de acciones discretizadas y inicialización de la Q-table
#### En este bloque se define el número de dimensiones del espacio de acciones (action_space_dim) del entorno. Luego, se crea action_bins, que divide el rango de posibles acciones de -1 a 1 en intervalos discretos, usando n_bins divisiones para cada dimensión. Finalmente, se inicializa la Q-table como un diccionario donde cada estado tiene un conjunto de acciones posibles, inicialmente todas con valor cero. Esto permitirá al algoritmo almacenar y actualizar las recompensas asociadas a cada acción en cada estado.

In [ ]:
# discretiza el espacio de observación y acción
action_space_dim = env.action_space.shape[0] # obtener la dimensión del espacio de acción
action_bins = np.linspace(-1, 1, n_bins)  # Rango de acción para cada articulación

# Inicializar Q-table como un diccionario
q_table = defaultdict(lambda: np.zeros((n_bins,) * action_space_dim))

### Definición de la política de acción (ε-greedy)
#### La función choose_action implementa la política ε-greedy para seleccionar acciones. Primero, discretiza el estado actual usando la función discretize_obs para convertirlo en una clave que puede ser utilizada en la Q-table. Si el agente está en modo de entrenamiento y se cumple una probabilidad definida por epsilon, selecciona una acción aleatoria en cada dimensión del espacio de acción. Si no, elige la mejor acción disponible en cada dimensión, buscando la acción con el valor más alto en la Q-table para el estado dado.

In [ ]:
# Definir la política de acción (ε-greedy)
def choose_action(state, training=True):
    # Discretizar el estado para usarlo como clave
    state = discretize_obs(state)

    if training and np.random.rand() < epsilon: # Si el numero aleatorio es menor a epsilon se explora
        # Acción aleatoria en cada dimensión
        return np.random.choice(action_bins, size=action_space_dim) 
    else: # Si no se explota
        # Seleccionar la mejor acción para cada dimensión
        best_action_idx = np.unravel_index(np.argmax(q_table[state]), (n_bins,) * action_space_dim) 
        return np.array([action_bins[i] for i in best_action_idx])

### Entrenamiento del agente con Q-learning
#### Este bloque entrena al agente utilizando Q-learning. En cada episodio, el agente selecciona acciones según la política ε-greedy, actualiza la Q-table usando la ecuación de Q-learning y acumula las recompensas obtenidas. Al final de cada episodio, se guarda la recompensa total, y cada 100 episodios se imprime el progreso. Finalmente, se grafica la recompensa promedio para mostrar el aprendizaje del agente a lo largo del tiempo.

In [ ]:
# Entrenar el agente con Q-learning
rewards_per_episode = [] # definir una lista para guardar las recompensas por episodio

for episode in range(5000):
    state, _ = env.reset() # Reiniciar el entorno
    done = False # Bandera para indicar si el episodio ha terminado
    total_reward = 0 # Inicializar la recompensa acumulada

    while not done: # Mientras el episodio no haya terminado
        action = choose_action(state) # Elegir una acción
        next_state, reward, terminated, truncated, _ = env.step(action) # Tomar la acción y obtener el siguiente estado y la recompensa
        done = terminated or truncated # El episodio termina si el entorno lo indica

        # Discretizar el estado y la acción
        state_discrete = discretize_obs(state) # Discretizar el estado actual
        next_state_discrete = discretize_obs(next_state) # Discretizar el siguiente estado
        action_idx = tuple(np.digitize(a, action_bins) - 1 for a in action) # Discretizar la acción tomada

        # Actualizar Q-table usando la ecuación de Q-learning
        if not done: # Si el episodio no ha terminado
            next_action_idx = np.unravel_index(np.argmax(q_table[next_state_discrete]), (n_bins,) * action_space_dim) # Mejor acción para el siguiente estado
            best_next_q_value = q_table[next_state_discrete][next_action_idx] # Mejor valor Q para el siguiente estado
            td_target = reward + gamma * best_next_q_value # Calcular el target de TD
        else: # Si el episodio ha terminado
            td_target = reward # El target de TD es la recompensa inmediata

        td_error = td_target - q_table[state_discrete][action_idx] # mide la diferencia entre el valoer actual y el objetivo
        q_table[state_discrete][action_idx] += alpha * td_error # Actualizar la Q-table

        # Acumular la recompensa
        total_reward += reward # Acumular la recompensa
        state = next_state # Actualizar el estado actual

    # Guardar la recompensa total del episodio
    rewards_per_episode.append(total_reward)

    # Mostrar progreso
    if episode % 100 == 0:
        print(f"Episodio {episode}, Recompensa total: {total_reward}")

# Graficar las recompensas por episodio
window_size = 100
smoothed_rewards = np.convolve(rewards_per_episode, np.ones(window_size) / window_size, mode='valid')
plt.plot(smoothed_rewards)
plt.xlabel('Episodios')
plt.ylabel('Recompensa Promedio (Promedio móvil)')
plt.title('Progreso del Aprendizaje del Agente en BipedalWalker-v3')
plt.show()

Episodio 0, Recompensa total: -106.65746367270127
Episodio 100, Recompensa total: -100.70155690841253
Episodio 200, Recompensa total: -132.17910480513422
Episodio 300, Recompensa total: -121.74853059161826
Episodio 400, Recompensa total: -127.38044910318591
Episodio 500, Recompensa total: -121.64355758916216
Episodio 600, Recompensa total: -102.7134151282354
Episodio 700, Recompensa total: -127.46389534347007
Episodio 800, Recompensa total: -104.45336570256514
Episodio 900, Recompensa total: -101.72573793094915
Episodio 1000, Recompensa total: -71.11057206868854
Episodio 1100, Recompensa total: -105.05328476859691
Episodio 1200, Recompensa total: -48.8864116502744
Episodio 1300, Recompensa total: -58.69774413249269
Episodio 1400, Recompensa total: -104.44705852873747
Episodio 1500, Recompensa total: -102.90411899704549
Episodio 1600, Recompensa total: -122.81210711845321
Episodio 1700, Recompensa total: -100.83693448058143
Episodio 1800, Recompensa total: -118.29568583349202
Episodio 1

### Muestra de la Tabla Q
#### En esta sección, se presenta una muestra de los primeros estados y sus valores Q correspondientes. La tabla Q almacena los valores de recompensa esperada para cada acción en un estado específico. Cada entrada en la tabla representa un estado del entorno y un conjunto de valores Q para las acciones posibles que el agente puede tomar. Estos valores indican cuán beneficiosa es una acción en un estado dado, lo que guía las decisiones del agente.

In [ ]:
# Mostrar una parte de la Tabla Q
print("\n--- Muestra de la Tabla Q ---")
sample_states = list(q_table.keys())[:5]  # Muestra de los primeros 5 estados
for state in sample_states: # Mostrar los valores Q para cada estado
    print(f"Estado {state}: Q-valores {q_table[state]}") 


--- Muestra de la Tabla Q ---
Estado (np.int64(3), np.int64(2), np.int64(3), np.int64(2), np.int64(3), np.int64(2), np.int64(3), np.int64(3), np.int64(3), np.int64(3), np.int64(2), np.int64(3), np.int64(3), np.int64(3), np.int64(3), np.int64(3), np.int64(3), np.int64(3), np.int64(3), np.int64(3), np.int64(3), np.int64(3), np.int64(3), np.int64(3)): Q-valores [[[[-0.02977647 -0.02865672 -0.02502159 -0.01906925 -0.0206307 ]
   [-0.02420899 -0.02277553 -0.0222749  -0.01773057 -0.01923309]
   [-0.01882019 -0.01730411 -0.01629436 -0.01760114 -0.01893981]
   [-0.01601012 -0.01474611 -0.01316905 -0.01542315 -0.01612064]
   [-0.02087623 -0.01949129 -0.01403491 -0.01822451 -0.01344243]]

  [[-0.02867429 -0.02768022 -0.02337363 -0.01772842 -0.01907278]
   [-0.02325837 -0.02182003 -0.02096391 -0.01637428 -0.01762305]
   [-0.01762809 -0.01629185 -0.01518452 -0.01616429 -0.01752308]
   [-0.01489437 -0.01351638 -0.0120553  -0.01406431 -0.01484764]
   [-0.01859701 -0.01730455  0.          0.        

In [ ]:
import numpy as np # type: ignore
import pandas as pd # type: ignore

# Muestra de la Tabla Q mejorada con pandas
sample_states = list(q_table.keys())[:5]  # Muestra de los primeros 5 estados

# Crear una lista para almacenar los datos
data = []

# Redondear valores usando numpy
for state in sample_states:
    q_values = q_table[state]
    q_values_rounded = np.round(q_values, 3)  # Redondear todo el arreglo a 3 decimales
    data.append({'Estado': state, 'Q-valores': q_values_rounded.tolist()})  # Convertir a lista para mejor visualización

# Crear DataFrame
df = pd.DataFrame(data)


In [ ]:
df

,Estado,Q-valores
0,"(3, 2, 3, 2, 3, 2, 3, 3, 3, 3, 2, 3, 3, 3, 3, ...","[[[[-0.03, -0.029, -0.025, -0.019, -0.021], [-..."
1,"(3, 3, 3, 2, 3, 3, 3, 2, 3, 3, 3, 3, 2, 3, 3, ...","[[[[-0.041, -0.039, -0.035, -0.03, -0.032], [-..."
2,"(3, 3, 3, 2, 3, 2, 2, 2, 3, 3, 2, 2, 2, 3, 3, ...","[[[[-0.04, -0.038, -0.036, -0.033, -0.033], [-..."
3,"(3, 3, 3, 2, 3, 2, 2, 2, 3, 2, 2, 2, 2, 3, 3, ...","[[[[-0.038, -0.038, -0.035, -0.032, -0.025], [..."
4,"(3, 3, 3, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 3, 3, ...","[[[[-0.038, -0.034, -0.031, -0.031, -0.036], [..."


In [ ]:
num_estados = len(q_table)
print(f"Número de estados en la Q-table: {num_estados}")


Número de estados en la Q-table: 424


### Prueba del Agente en Modo Explotación
#### En este bloque de código, se realiza una prueba del agente en modo explotación, donde el agente toma decisiones basadas únicamente en su tabla Q, sin explorar acciones aleatorias. En cada episodio, el agente elige la acción con la mayor recompensa esperada en el estado actual. El ciclo se repite hasta que el episodio termine, acumulando la recompensa total. Al final, se imprime la recompensa obtenida por episodio y el promedio general de las recompensas en todas las pruebas, lo que permite evaluar el desempeño del agente en el entorno.

In [ ]:
# Prueba del agente en modo explotación (sin exploración)
def test_agent(num_episodes=10):
    epsilon = 0  # Desactivar exploración
    total_rewards = []
    
    # Crear el entorno con render_mode="human" solo para la fase de prueba
    env = gym.make("BipedalWalker-v3", render_mode="human")
    
    for episode in range(num_episodes):
        state, _ = env.reset()
        done = False
        total_reward = 0
        while not done:
            action = choose_action(state, training=False)
            next_state, reward, terminated, truncated, _ = env.step(action)
            total_reward += reward
            state = next_state
            done = terminated or truncated
            env.render()  # Mostrar el render del entorno en cada paso (solo en explotación)
        total_rewards.append(total_reward)
        print(f"Recompensa total en episodio de prueba {episode + 1}: {total_reward}")
    print(f"\nRecompensa promedio en modo explotación: {np.mean(total_rewards)}")

# Ejecutar la prueba del agente
test_agent()


Recompensa total en episodio de prueba 1: -102.97181909203468
Recompensa total en episodio de prueba 2: -117.01173308462587
Recompensa total en episodio de prueba 3: -121.78440617527254
Recompensa total en episodio de prueba 4: -100.42383098423977
Recompensa total en episodio de prueba 5: -128.74028710572483
Recompensa total en episodio de prueba 6: -126.54417246946133
Recompensa total en episodio de prueba 7: -127.93529670975728
Recompensa total en episodio de prueba 8: -125.00364205226549
Recompensa total en episodio de prueba 9: -126.51157220281473
Recompensa total en episodio de prueba 10: -103.40621052407411

Recompensa promedio en modo explotación: -118.03329704002707


In [ ]:
env.close()

: 